In [13]:
from PIL import Image 
import numpy as np 
import nrrd# Some sample numpy 
import os,glob
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import cv2,math
import matplotlib as mpl
import requests
import pandas as pd
from tqdm import tqdm

In [7]:
# download the nrrd file from allen institute
def download_file(url, file_path):
    response = requests.get(url)
    with open(file_path, 'wb') as f:
        f.write(response.content)

file_name = 'annotation_10.nrrd'
if not os.path.exists(file_name):
    url = 'http://download.alleninstitute.org/informatics-archive/current-release/mouse_ccf/annotation/ccf_2017/' + file_name
    download_file(url, file_name)
    print(f"File '{file_name}' downloaded successfully.")
else:
    print(f"File '{file_name}' already exists.")

File 'annotation_10.nrrd' downloaded successfully.


In [14]:
dfarea = pd.read_excel("brain_areas.xlsx",index_col=0)

In [15]:
retain_area = ["ARH","V3"]
retain_ID = dfarea.loc[dfarea.region.isin(retain_area)].index.tolist()

In [16]:
somadata,header=nrrd.read("annotation_10.nrrd")
mask = ~np.isin(somadata,retain_ID+[0])
somadata[mask] = 5

In [17]:
dfplot = pd.read_excel("Agrp_Pomc_ARH_soma_infor.xlsx",index_col=0)

In [19]:
# plot soma location in the allen image
bregma = True
hue_para = "geno"
space = 400

dic_list = {'Pomc':'#F9CE00','Agrp':'#0555FF'}

plot_max = math.ceil(dfplot.AP.max()/10)
plot_min = int(dfplot.AP.min()/10)
print(plot_max)
print(plot_min)
try:
    os.mkdir("raw")
except:
    print("The folder already exist")
for start in range(plot_min,plot_max,int(space/10)):
    # image = somadata[int(start),:,:].astype("uint8")
    image = somadata[int(start+space/30),:,:].astype("uint8")
    edges = cv2.Canny(image,10,25)
    edges2 = 255-edges
    dfnew = dfplot.loc[(dfplot.AP>start*10)&(dfplot.AP<start*10+space)]
    fig = plt.figure(figsize=(16,8),dpi=600)
    ax = fig.add_subplot(1,1,1)
    ax.imshow(edges2,cmap = 'gray')
    for sep_i in dic_list:   
        dfsubplot = dfnew.loc[dfnew.loc[:,hue_para] == sep_i]
        ax.scatter(dfsubplot.ML/10,dfsubplot.DV/10,s=4,c=dic_list[sep_i],marker = "o",alpha = 1)  
    ax.axis("off")

    if bregma == False:
        plt.savefig(".\\raw\\soma_%d.jpg"%int(start+space/30),dpi=900)
    else:
        plt.savefig(".\\raw\\soma_%d.jpg"%int(-(start*10+space/3-5472)/100),dpi=900)
    plt.close()   

744
651
The folder already exist


In [23]:
os.mkdir("crop")

In [20]:
# ARH
para = (6900,5100,7410,6000)
mirrow = False

In [21]:
filelist = glob.glob(".\\raw\\*")

In [22]:
# crop image
for file in filelist:
    filetmp = Image.open(file).crop(para)
    # filetmp = Image.open(file)
    filename = file.split("\\")[-1]
    # filetmp = filetmp.rotate(-90)
    if mirrow == True:
        filetmp = filetmp.transpose(Image.FLIP_LEFT_RIGHT)
        filetmp = filetmp.rotate(-90)
    filetmp.save(os.path.join(".\\crop",filename))
    filetmp.close()

C:\Users\win7\AppData\Local\Programs\Python\Python37\Lib\site-packages\PIL\Image.py:3170: DecompressionBombWarning: Image size (103680000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning,


In [39]:
os.mkdir("edge")

FileExistsError: [WinError 183] 当文件已存在时，无法创建该文件。: 'edge'

In [23]:
filelist = glob.glob(".\\crop\\*")

In [24]:
# add edge
add = 8
for file in filelist:
    tmp1st = Image.open(file)
    x = int(tmp1st.size[0]+add*2)
    y = int(tmp1st.size[1]+add*2)
    result = Image.new(tmp1st.mode, (x,y),"black")
    # result = Image.new(tmp1st.mode, (x,y),"white")
    result.paste(tmp1st,box=(add,add))
    result.save(".\\edge\\%s"%(file.split("\\")[-1]))
    result.close()
